# TFTrade Model Experiments 


__Financial Prediction research__


Research about financial prediction on markets in MetaTrader 5.
Deriv Synthetic instruments are currently used as the primary instruments.

**NOTE:** Docker is required to run this notebook.



In [5]:
# !pip install pandas-ta yfinance docker python-dotenv

## Start MetaTrader 5 Docker Terminal

In [6]:
import logging
import time
from typing import Optional
from trade_flow.common.logging import Logger
from packages.mt5any import (
    DockerizedMT5TerminalConfig,
    DockerizedMT5Terminal,
)
from packages.mt5any import MetaTrader5


class MT5Exception(Exception):
    """Base exception class for MT5 errors."""

    pass


class MT5InitializationError(MT5Exception):
    """Raised when there is an error initializing MT5 Trader."""

    pass


class MT5:
    """
    A class to interface with MetaTrader 5 for trading operations using Dockerized MT5 Terminals.

    Attributes:
        mt5_account_number (str): The MetaTrader 5 account number.
        mt5_password (str): The password for the MT5 account.
        mt5_server (str): The MT5 server name.
        logger (Logger): Logger instance for logging events.
        mt5_terminal (DockerizedMT5Terminal): Dockerized MT5 Terminal instance.
        mt5 (MetaTrader5): MetaTrader 5 client interface.
        initial_balance (float): The initial account balance.
    """

    def __init__(
        self,
        account_number: str,
        password: str,
        server: str,
        logger: Optional[Logger] = None,
    ) -> None:
        """
        Initialize the MT5 class with account credentials and configurations.

        Args:
            account_number (str): MetaTrader 5 account number.
            password (str): MetaTrader 5 account password.
            server (str): MetaTrader 5 server.
            logger (Optional[Logger]): Logger instance for logging.
        """
        self.mt5_account_number = account_number
        self.mt5_password = password
        self.mt5_server = server

        # Set up logging
        self.logger = logger or Logger(name="tft_logger", log_level=logging.DEBUG, filename="TFT_model_experiments.log")

        # Set up MetaTrader 5 terminal and configuration
        self.mt5_config = DockerizedMT5TerminalConfig(
            account_number=self.mt5_account_number,
            password=self.mt5_password,
            server=self.mt5_server,
            read_only_api=True,
        )

        # Initialize Dockerized MT5 Terminal
        self.mt5_terminal = DockerizedMT5Terminal(config=self.mt5_config)
        self._initialize_terminal()

        # Initialize MetaTrader 5
        self.mt5 = MetaTrader5()
        self._initialize_mt5()
        self.logger.debug(f"Terminal Info: {self.mt5.terminal_info()._asdict()}")

        # Get account information
        self.account_info = self.mt5.account_info()._asdict()
        self.logger.debug(f"Account Info: {self.account_info}")

        self.initial_balance = self.account_info["balance"]

        # Log account info
        self.logger.info(f"Account Balance: {self.initial_balance}")
        self.logger.info(f"Equity: {self.account_info['equity']}")
        self.logger.info(f"Currency: {self.account_info['currency']}")
        self.logger.info(f"Margin: {self.account_info['margin']}")
        self.logger.info(f"Server: {self.account_info['server']}")
        self.logger.info(f"Name: {self.account_info['name']}")

    def _initialize_terminal(self) -> None:
        """Initialize and safely start the Dockerized MT5 Terminal."""
        try:
            self.mt5_terminal.safe_start()
            time.sleep(5)

            self.logger.info(f"MetaTrader 5 Terminal started for account {self.mt5_account_number}")
        except Exception as e:
            self.logger.error(f"Error initializing Dockerized MT5 Terminal: {e}")
            raise MT5InitializationError("Failed to start Dockerized MT5 Terminal")

    def _initialize_mt5(self) -> None:
        """Initialize the MetaTrader 5 client."""
        try:
            if not self.mt5.initialize():
                raise RuntimeError("MetaTrader 5 initialization failed")

            # if not self.mt5.login(self.mt5_account_number, self.mt5_password, self.mt5_server):
            #     raise RuntimeError("MetaTrader 5 login failed")
        except Exception as e:
            self.logger.error(f"Error initializing MetaTrader 5: {e}")
            raise MT5InitializationError("Failed to initialize MetaTrader 5")


In [ ]:
import os 
from dotenv import load_dotenv

load_dotenv()

mt5_venue = MT5(
    account_number=os.getenv("MT5_ACCOUNT_NUMBER"),
    password=os.getenv("MT5_PASSWORD"),
    server=os.getenv("MT5_SERVER"),
)


In [ ]:
mt5_venue

## Import Libraries

In [5]:
# base import
import os
import sys
import json
import yaml
import time
import random 
import numpy as np
import pandas as pd
import pandas_ta as ta
import sklearn as sk
import seaborn as sns
import matplotlib as matplot
import matplotlib.pyplot as plt

from typing import List, Tuple
from datetime import datetime, timedelta

import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer

from IPython.display import clear_output
# import jtplot module in notebook
# from jupyterthemes import jtplot

# choose which theme to inherit plotting style from
# onedork | grade3 | oceans16 | chesterish | monokai | solarizedl | solarizedd
# jtplot.style(theme='solarizedd')
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [6]:
# plt.style.available
plt.style.use('seaborn-v0_8-whitegrid')

In [7]:
def set_seeds(seed=100):
    random.seed(seed)          # Set seed for Python's random module
    np.random.seed(seed)       # Set seed for NumPy's random module
    torch.manual_seed(seed)    # Set seed for PyTorch
    torch.cuda.manual_seed(seed)  # Set seed for CUDA if using GPU
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior for cuDNN
    torch.backends.cudnn.benchmark = False      # Disable the benchmark for deterministic results

set_seeds()

## Library Version and Docs

In [ ]:
print("Library version:")
print("\t-  pandas: {}".format(pd.__version__))
print("\t-  numpy: {}".format(np.__version__))
print("\t-  sklearn: {}".format(sk.__version__))
print("\t-  torch: {}".format(torch.__version__))
print("\t-  matplotlib: {}".format(matplot.__version__))
print("\t-  python: {}".format(sys.version[:sys.version.find('(') - 1]))

In [ ]:
df = pd.DataFrame()

df.ta.indicators()

In [ ]:
help(ta.sma)

## Load data

### Initialize MetaTrader 5

In [ ]:
mt5 = mt5_venue.mt5

# INITIALIZE THE DEVICE
mt5.initialize()


### Fetch All Available Symbols

In [ ]:
# Create empty lists
symbols = []
sectors = []
descriptions = []

# Get the information for all symbol
symbols_information = mt5.symbols_get()

symbols_information

In [ ]:
len(symbols_information)

### Extract Symbols Information

In [ ]:
symbols = [element[-3] for element in symbols_information]
sectors = [element[-1].split("\\")[0] for element in symbols_information]
descriptions = [element[-7] for element in symbols_information]

# Create a dataframe directly from the extracted lists
informations = pd.DataFrame({
    "Symbol": symbols,
    "Sector": sectors,
    "Description": descriptions
})

informations

**Remove Forex Pairs**

In [ ]:
filtered_Informations = informations[~informations['Sector'].str.contains("Forex")]

filtered_Informations

**Take some Volatility pairs**

In [16]:
# Filter out rows where 'Symbol' does not contains 'Volatility'
# filter_volatility_pairs = informations[informations['Symbol'].str.contains("Volatility")]

# filter_volatility_pairs

### Select Symbols 

In [ ]:
# selected_df = pd.concat([lowest_spread_asset, filter_volatility_pairs])
selected_df = filtered_Informations

selected_df

**Select the symbols on MT5**

In [ ]:
# Loop through the selected symbols and select them in MetaTrader5
for symbol in selected_df['Symbol']:
    # Try to select the symbol, raise an error if it fails
    if not mt5.symbol_select(symbol, True):
        raise ValueError(f"Failed to enable symbol: {symbol}")
    else:
        print(f"Symbol enabled: {symbol}")

### Define Get data function

In [19]:
import pytz
import pandas as pd
from datetime import datetime, timedelta

def get_data(symbol: str, n: int, timeframe=mt5.TIMEFRAME_D1, **kwargs) -> pd.DataFrame:
    """
    Fetch historical data for a given symbol from MetaTrader 5.

    This function retrieves data for a specified symbol either for a certain range of time 
    or a specific number of bars. It ensures that the symbol is enabled in MetaTrader 5 and 
    handles both custom time ranges and default time periods.

    Args:
        symbol (str): The trading symbol (e.g., 'EURUSD').
        n (int): Number of bars (data points) to retrieve if not using a time range.
        timeframe (int): The time frame to use (e.g., mt5.TIMEFRAME_D1 for daily). Default is mt5.TIMEFRAME_D1.
        **kwargs: Additional arguments:
            - is_range (bool): Whether to fetch data for a specific time range.
            - utc_from (datetime): Start time (UTC) for the time range (default: 2020-01-10).
            - utc_to (datetime): End time (UTC) for the time range (default: 2020-01-11 13:00).

    Returns:
        pd.DataFrame: DataFrame containing historical rates with 'time' as the index.

    Raises:
        ValueError: If the symbol cannot be enabled in MetaTrader 5.

    Example Usage:
        - To fetch 1000 daily bars:
            >>> df = get_data('EURUSD', 1000)

        - To fetch data within a specific time range:
            >>> df = get_data('EURUSD', 0, is_range=True, utc_from=datetime(2022, 1, 1), utc_to=datetime(2022, 1, 10))
    """
    
    # Initialize MetaTrader 5 terminal
    mt5.initialize()

    # Select symbol and ensure visibility
    if not mt5.symbol_select(symbol, True):
        raise ValueError(f"Failed to enable symbol: {symbol}")

    # Handle data retrieval based on range or last 'n' bars
    is_range = kwargs.get("is_range", False)
    timezone = pytz.timezone("Etc/UTC")  # UTC timezone

    if is_range:
        # Fetch data in a specific time range
        utc_from = kwargs.get("utc_from", datetime(2020, 1, 10, tzinfo=timezone))
        utc_to = kwargs.get("utc_to", datetime(2020, 1, 11, hour=13, tzinfo=timezone))
        rates = mt5.copy_rates_range(symbol, timeframe, utc_from, utc_to)
    else:
        # Fetch the last 'n' bars
        utc_from = datetime.now() + timedelta(hours=2)
        rates = mt5.copy_rates_from(symbol, timeframe, utc_from, n)

    # Convert the rates into a DataFrame
    rates_frame = pd.DataFrame(rates)

    # Convert 'time' from seconds to datetime format and set as index
    rates_frame['time'] = pd.to_datetime(rates_frame['time'], unit='s')
    # rates_frame['time'] = pd.to_datetime(rates_frame['time'], format='%Y-%m-%d')
    rates_frame.set_index('time', inplace=True)

    return rates_frame


## Find the Best Spread Threshold

In [ ]:
# Create a list to store the spread values
spread = []

# Compute the spread using a list comprehension for efficiency
spread = [
    (tick.ask - tick.bid) / tick.bid if (tick := mt5.symbol_info_tick(symbol)) and tick.ask and tick.bid else None
    for symbol in selected_df["Symbol"]
]

spread

In [ ]:
min_spread = min([x for x in spread if x is not None])

min_spread

In [ ]:
max_spread = max([x for x in spread if x is not None])

max_spread

In [ ]:
mean_spread = np.mean([min_spread, max_spread])

mean_spread

In [ ]:
mean_spread/100

In [ ]:
# if mean_spread < 0.01:
#     spread_threshold = np.round(mean_spread * 100, 4) # 0.0035
# else:
#     spread_threshold = mean_spread

spread_threshold = np.round(mean_spread / 1000, 4) # 0.0035
spread_threshold

#### Take the assets with low spreads

In [ ]:
selected_df["Spread"] = spread

# Take the assets with the spread < spread_threshold(%)
lowest_spread_asset = selected_df.dropna().loc[selected_df["Spread"]<spread_threshold]
lowest_spread_asset

## Select Final Assets 

In [ ]:
final_assets = lowest_spread_asset
# final_assets = lowest_spread_asset.head(10)
# Fill Assets with no spread with max spread value
# final_assets['Spread'] = final_assets['Spread'].fillna(max_spread) 

final_assets

## Features engineering 


### Define Normalization Scheme

This class apply logarithm and a normalization(MinMax | StandarScale | Normalizer_l1 | Normalizer_l2).

In [28]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer as SklearnNormalizer

class Normalizer:
    """
    A class for normalizing market data while preserving the original data.

    This class applies logarithm and a normalization (MinMax, StandardScale, Normalizer_l1, Normalizer_l2).
    """

    def __init__(self, data):
        """
        Initializes the Normalizer class with market data.

        Args:
            data (list[dict]): A list of dictionaries, where each dictionary represents a market
                               with keys like "name" and "data" (containing market data).
        """
        self.__market_names = [e["name"] for e in data]
        self.__original_data = [pd.DataFrame(e["data"]) for e in data]  # Convert to DataFrame
        self.__normalized_data = None  # Stores normalized data after fit is called

    def __normalize(self, target, numerical_data, scaler_func):
        """
        Normalizes data using the provided scaler function for specific target markets.

        Args:
            target (str): The target market(s) to normalize ("all" or a list of market names).
            numerical_data (list[pd.DataFrame]): A list of DataFrames containing numerical data for each market.
            scaler_func (callable): A function that performs data scaling (e.g., MinMaxScaler.fit_transform).

        Returns:
            list[pd.DataFrame]: List of normalized market DataFrames.
        """
        normalized_markets = []
        target_markets = self.__market_names if target == "all" else target

        for market_name, market in zip(self.__market_names, numerical_data):
            if market_name in target_markets:
                columns = market.columns
                market_scaled = scaler_func(market)
                normalized_markets.append(pd.DataFrame(market_scaled, columns=columns))

        return normalized_markets

    def fit(self, norm_type, features_list, target="all"):
        """
        Performs data normalization based on the specified type, features, and target market(s).

        Args:
            norm_type (str): The type of normalization to perform ("MinMax", "StandardScale", "Normalizer_l1", or "Normalizer_l2").
            features_list (list): A list of feature names to consider for normalization.
            target (str, optional): The target market(s) to normalize ("all" or a list of market names). Defaults to "all".

        Raises:
            ValueError: If an invalid normalization type is provided.
        """
        # Extract numerical data for the specified features
        numerical_data = [market[features_list]._get_numeric_data() for market in self.__original_data]

        if norm_type == "MinMax":
            print(f"Performing MinMax Normalization on {target}.")
            scaler_func = MinMaxScaler().fit_transform
        elif norm_type == "StandardScale":
            print(f"Performing StandardScale Normalization on {target}.")
            scaler_func = StandardScaler().fit_transform
        elif norm_type.startswith("Normalizer"):
            norm_value = norm_type.split("_")[-1]  # Extract l1 or l2 from "Normalizer_l1" or "Normalizer_l2"
            if norm_value not in ("l1", "l2"):
                raise ValueError("Invalid norm type for Normalizer. Must be 'l1' or 'l2'.")
            print(f"Performing Normalizer (norm={norm_value}) on {target}.")
            scaler_func = SklearnNormalizer(norm=norm_value).fit_transform
        else:
            raise ValueError(f"Invalid normalization type: {norm_type}")

        self.__normalized_data = self.__normalize(target, numerical_data, scaler_func)

    def get_normalized_data(self, idx: int = None):
        """
        Returns the normalized data if normalization has been performed, otherwise raises an error.
        
        Args:
            idx (int, optional): The location/index of the normalized data. If None, returns all normalized data.

        Raises:
            RuntimeError: If data has not been normalized yet.
            IndexError: If the index is out of range.
        """
        if self.__normalized_data is None:
            raise RuntimeError("Data has not been normalized yet. Please call 'fit' first.")
        
        if idx is not None:
            if idx < 0 or idx >= len(self.__normalized_data):
                raise IndexError("Index out of range.")
            return self.__normalized_data[idx]
        
        return self.__normalized_data  # Return all normalized data if idx is None

    def get_original_data(self):
        """
        Returns the original, un-normalized data.
        """
        return [data.copy() for data in self.__original_data]  # Return a deep copy


### Define PEAKS Detection 

This function detect peaks with a delta.

Choosing a default delta value for the peakdet function depends on the characteristics of your data and the desired level of peak/valley sensitivity. Here are some considerations:

- Data Scale: If your data values are on a large scale (e.g., stock prices in the thousands), a larger delta might be appropriate to avoid detecting insignificant fluctuations.
- Noise Level: If your data has a high level of noise, a larger delta might be necessary to filter out minor variations and focus on more prominent peaks and valleys.
- Desired Sensitivity: If you want to capture a broad range of peaks and valleys, a smaller delta would be suitable. However, this might also lead to detecting insignificant pumps or dips.
  
Here are some possible default values based on common scenarios:

- General Case: A reasonable starting point for many applications could be a delta value between 0.01 and 0.1. This range is a relative percentage of the data scale and can capture significant peaks and valleys without being overly sensitive to noise.
- Highly Scaled Data (e.g., Stock Prices): You might consider a delta between 1.0 and 10.0 for data with large values.
- Noisy Data: A delta between 0.05 and 0.2 could be a starting point for data with significant noise.

In [29]:

def peaks_detection(data: list[float], delta: float = 0.01, x: list[float] = None) -> tuple[np.ndarray, np.ndarray]:
    """
    Finds peaks and valleys in a data series.

    Args:
        data (list[float]): The data series.
        delta (float): The threshold for a peak or valley.
        x (list[float], optional): The x-axis values (optional). Defaults to None.

    Returns:
        tuple[np.ndarray, np.ndarray]: Two numpy arrays, the first containing the indices and values of the peaks,
               the second containing the indices and values of the valleys.
    """

    data_array = np.asarray(data)  # Ensure NumPy array for efficiency

    if x is None:
        x = np.arange(len(data_array))  # Create x-axis if not provided

    peaks: list[tuple[float, float]] = []
    valleys: list[tuple[float, float]] = []
    current_peak = np.inf
    current_valley = -np.inf
    peak_pos = np.nan
    valley_pos = np.nan
    looking_for_peak = True  # Flag to track search direction

    for i, this_value in enumerate(data_array):
        # Update current peak and valley values
        current_peak = max(current_peak, this_value)
        current_valley = min(current_valley, this_value)

        if looking_for_peak:
            if this_value < current_peak - delta:
                if not np.isnan(peak_pos):
                    peaks.append((x[int(peak_pos)], current_peak))
                # peaks.append((x[int(peak_pos)], current_peak))
                current_valley = this_value
                valley_pos = i
                looking_for_peak = False
        else:
            if this_value > current_valley + delta:
                if not np.isnan(valley_pos):
                    valleys.append((x[int(valley_pos)], current_valley))
                # valleys.append((x[int(valley_pos)], current_valley))
                current_peak = this_value
                peak_pos = i
                looking_for_peak = True

    return np.array(peaks), np.array(valleys)


### Define Frame labelization 

Currently: buy, sell, wait/neutral

update to: strong buy, buy, strong sell, sell, wait/neutral (can also use the prediction probability to determine this 
e.g when a buy is >=75%, it can indicate a strong buy)

can also rename wait to neutral 


In [30]:
class MarketLabeler:
    """
    A class for labeling market data with buy, sell, or neutral signals based on peak information.
    """

    def __init__(self, peaks_max, peaks_min):
        """
        Initializes the MarketLabeler with peak data.

        Args:
            peaks_max (list): A list of tuples representing peak maxima (index, value).
            peaks_min (list): A list of tuples representing peak minima (index, value).
        """
        self.peaks_max = peaks_max
        self.peaks_min = peaks_min

    def label_data(self, data):
        """
        Labels each data point with a buy, sell, or neutral signal based on peaks.

        Args:
            data (list): A list representing the market data (e.g., prices).

        Returns:
            list: A list of labels (0 for neutral, 1 for sell, 2 for buy) corresponding to each data point.
        """
        labels = []
        for i, _ in enumerate(data):  # Iterate through data indices
            if any(idx == i for idx, _ in self.peaks_min):
                labels.append(0)  # Buy signal if peak min
            elif any(idx == i for idx, _ in self.peaks_max):
                labels.append(1)  # Sell signal if peak max
            else:
                labels.append(2)  # Neutral otherwise
        return labels

    def label_dataframe(self, frame_base: pd.DataFrame, data_column: str = "close"):
        """
        Labels a DataFrame with buy, sell, and neutral columns based on peak information.

        Args:
            frame_base (pd.DataFrame): The DataFrame containing the market data.
            data_column (str, optional): The name of the column containing the data to be labeled. Defaults to "close".

        Returns:
            pd.DataFrame: The updated DataFrame with added "buy", "sell", and "neutral" columns.
        """
        if data_column not in frame_base.columns:
            raise ValueError(f"Data column '{data_column}' not found in the DataFrame.")

        labels = self.label_data(frame_base[data_column])
        
        frame_base["buy"] = [1 if label == 0 else 0 for label in labels]
        frame_base["sell"] = [1 if label == 1 else 0 for label in labels]
        frame_base["neutral"] = [1 if label == 2 else 0 for label in labels]
    
        return frame_base



### Define Data Generator

In [31]:

class DataGenerator:
    """
    A class that generates data for training and testing.

    Args:
        dataset (pd.DataFrame): The input dataset.
        timestep (int): The number of timesteps to consider for each sample.
        xcols (List[str]): The column names to be used as input features.
        ycols (List[str]): The column names to be used as output labels.

    Methods:
        generate_data: Generates the input-output pairs for training and testing.
        balance_labelization: Balances the label distribution by removing excess neutral labels.
        train_test_split: Splits the data into training and testing sets.

    """

    def __init__(self, dataset: pd.DataFrame, timestep: int, xcols: List[str], ycols: List[str]):
        self.dataset = dataset
        self.timestep = timestep
        self.xcols = xcols
        self.ycols = ycols

    def generate_data(self) -> Tuple[np.ndarray, np.ndarray]:
        """
        Generates the input-output pairs for training and testing.

        Returns:
            Tuple[np.ndarray, np.ndarray]: A tuple containing the input and output arrays.

        """
        dx = [np.array(self.dataset.iloc[i : i + self.timestep][self.xcols]) for i in range(len(self.dataset) - self.timestep)]
        dy = [self.dataset.iloc[i + self.timestep - 1][self.ycols] for i in range(len(self.dataset) - self.timestep)]
        return np.array(dx), np.array(dy)

    def balance_labelization(self, frame: np.ndarray, label: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """
        Balances the label distribution by removing excess neutral labels.

        Args:
            frame (np.ndarray): The input array.
            label (np.ndarray): The label array.

        Returns:
            Tuple[np.ndarray, np.ndarray]: A tuple containing the balanced input and label arrays.

        """
        neutral_indices = np.where(label[:, 2] == 1)[0]
        sell_indices = np.where(label[:, 1] == 1)[0]
        buy_indices = np.where(label[:, 0] == 1)[0]

        neutral_count = len(neutral_indices)
        sell_count = len(sell_indices)
        buy_count = len(buy_indices)

        need_delete = neutral_count - min(sell_count, buy_count)
        rand_delete = np.random.choice(neutral_indices, need_delete, replace=False)

        final_frame = np.delete(frame, rand_delete, axis=0)
        final_label = np.delete(label, rand_delete, axis=0)

        return final_frame, final_label

    def train_test_split(self, test_per: float, balance: bool = False) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """
        Splits the data into training and testing sets.

        Args:
            test_per (float): The percentage of data to be used for testing.
            balance (bool, optional): Whether to balance the label distribution. Defaults to False.

        Returns:
            Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]: A tuple containing the training and testing input and label arrays.

        """
        x_tmp, y_tmp = self.generate_data()

        if balance:
            x_tmp, y_tmp = self.balance_labelization(x_tmp, y_tmp)

        x_train, x_test, y_train, y_test = train_test_split(x_tmp, y_tmp, test_size=test_per, random_state = 100, shuffle = settings['model_shuffle'])

        return x_train.astype(np.float32), y_train.astype(np.float32), x_test.astype(np.float32), y_test.astype(np.float32)


###  Add available technical analysis (TA) features

In [32]:

def add_ta_features(df: pd.DataFrame) -> pd.DataFrame:
    """ 
    Adds various technical analysis (TA) features to the provided DataFrame.
    
    This function calculates several technical indicators, including momentum, 
    volatility, statistics, trend, volume, overlap, and performance indicators. 
    The input DataFrame must include a "date" column for setting the index to 
    a DatetimeIndex.

    References:
    - https://github.com/twopirllc/pandas-ta/blob/main/examples/AIExample.ipynb
    """
    
    # Create a copy of the DataFrame to avoid modifying the original
    features_df = df.copy() 

    # VWAP requires the DataFrame index to be a DatetimeIndex.
    # Replace "datetime" with the appropriate column from your DataFrame
    
    features_df.set_index(pd.DatetimeIndex(features_df.index), inplace=True)
    try:
        features_df.drop(columns=[features_df.index.name], inplace=True)
    except KeyError:
        print(f"Column '{features_df.index.name}' does not exist in the DataFrame.")
    except Exception as e:
        print(f"An error occurred while dropping the column: {e}")


    # Calculate Returns and append to the features_df DataFrame
    # features_df.ta.log_return(cumulative=True, append=True)
    # features_df.ta.percent_return(cumulative=True, append=True)

    """ 
        Add TA Indicators
    """
    
    # Adding momentum indicators
    features_df.ta.mom(append=True)
    features_df.ta.rsi(append=True)
    features_df.ta.tsi(append=True)
    features_df.ta.er(append=True)
    features_df.ta.fisher(append=True)

    # Adding volatility indicators
    features_df.ta.true_range(append=True)
    features_df.ta.rvi(append=True)
    features_df.ta.bbands(append=True)
    features_df.ta.pdist(append=True)

    # Adding statistics indicators
    features_df.ta.skew(append=True)
    features_df.ta.kurtosis(append=True)
    features_df.ta.mad(append=True)
    features_df.ta.zscore(append=True)
    features_df.ta.entropy(append=True)

    # Adding trend indicators
    features_df.ta.adx(append=True)
    features_df.ta.dpo(lookahead=False, append=True)
    features_df.ta.psar(append=True)
    features_df.ta.long_run(append=True)
    features_df.ta.short_run(append=True)
    features_df.ta.qstick(append=True)

    # Adding volume indicators
    # features_df.ta.obv(append=True)

    # Adding overlap indicators
    features_df.ta.linreg(append=True)
    features_df.ta.supertrend(append=True)
    features_df.ta.hilo(append=True)
    features_df.ta.hlc3(append=True)
    features_df.ta.ohlc4(append=True)

    # Adding Simple Moving Averages (SMA)
    sma_windows = [2, 10, 15, 60]
    for ma_window in sma_windows:
        features_df.ta.sma(length=ma_window, sma=False, append=True)

    # Adding Exponential Moving Averages (EMA)
    ema_windows = [8, 21, 50]
    for ma_window in ema_windows:
        features_df.ta.ema(length=ma_window, sma=False, append=True)

    # Adding performance indicators
    # features_df.ta.percent_return(append=True)

    print("TA Columns Added")

    """ 
    #### Modify RSI (Relative Strength Index) Indicator

        - The RSI provides technical traders with signals about bullish and bearish price momentum, 
        and it is often plotted beneath the graph of an asset’s price.

        - An asset is usually considered overbought when the RSI is above 70 and oversold when it is below 30.

        - The RSI line crossing below the overbought line or above the oversold line is often seen by traders 
        as a signal to buy or sell.

        - The RSI works best in trading ranges rather than trending markets.
    """
    features_df["RSI_14"] = features_df["RSI_14"].round()  # Rounding RSI values to the nearest integer

    # Add Candle Stick Patterns   
    features_df.ta.cdl_pattern(name="all", append=True)

    return features_df


### Define features engineering function

In [47]:
from sklearn.decomposition import PCA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer


def features_engineering(df: pd.DataFrame, add_ta: bool = False, compress_features: bool = True) -> dict:
    """ 
    This function creates the necessary datasets for algorithms 
    by performing feature engineering, scaling, and PCA.

    Args:
        df (DataFrame): The input dataframe containing price data (including 'close', 'high', 'low').
        add_ta (bool): Whether to add technical analysis features. Defaults to False.
        compress_features (bool): Whether to apply PCA for dimensionality reduction. Defaults to True.

    Returns:
        dict: A dictionary containing the train, test, validation sets, 
              along with their scaled and PCA-transformed versions.
    """

    original_df = df.copy()

    # Define the feature columns
    feature_columns = [
        "returns t-1", 
        "mean returns 15", 
        "mean returns 60",
        "volatility returns 15", 
        "volatility returns 60"
    ]
    
    # Create new columns for returns and other features

    # Ensure returns column is in the dataframe
    df["returns"] = (df["close"] - df["close"].shift(1)) / df["close"].shift(1)
    df["sLow"] = (df["low"] - df["close"].shift(1)) / df["close"].shift(1)
    df["sHigh"] = (df["high"] - df["close"].shift(1)) / df["close"].shift(1)

    # Feature engineering
    df["returns t-1"] = df["returns"].shift(1)
    df["mean returns 15"] = df["returns"].rolling(15).mean().shift(1)
    df["mean returns 60"] = df["returns"].rolling(60).mean().shift(1)
    df["volatility returns 15"] = df["returns"].rolling(15).std().shift(1)
    df["volatility returns 60"] = df["returns"].rolling(60).std().shift(1)

    # Add technical analysis features if specified
    if add_ta:
        df = add_ta_features(df)
        feature_columns = df.columns.to_list()

    # Impute missing values using IterativeImputer (Multiple Imputation by Chained Equations)
    # imputer = IterativeImputer(random_state=0)
    # df[feature_columns + ["returns"]] = imputer.fit_transform(df[feature_columns + ["returns"]])
    # df[feature_columns + ["returns"]] = df[feature_columns + ["returns"]].fillna(0)
    # print(df)

    # # Splitting data into train, test, and validation sets
    # split_train_test = int(0.70 * len(df))
    # split_test_valid = int(0.90 * len(df))

    # # Train set creation
    # X_train = df[feature_columns].iloc[:split_train_test]
    # y_train_reg = df["returns"].iloc[:split_train_test]
    # y_train_cla = np.round(df["returns"].iloc[:split_train_test] + 0.5)

    # # Test set creation
    # X_test = df[feature_columns].iloc[split_train_test:split_test_valid]
    # y_test_reg = df["returns"].iloc[split_train_test:split_test_valid]

    # # Validation set creation
    # X_val = df[feature_columns].iloc[split_test_valid:]
    # y_val_reg = df["returns"].iloc[split_test_valid:]

    # # Standardize the data
    # scaler = StandardScaler()
    # X_train_scaled = scaler.fit_transform(X_train)
    # X_test_scaled = scaler.transform(X_test)
    # X_val_scaled = scaler.transform(X_val)

    # Use the Normalizer for scaling
    # normalizer = Normalizer(data=[{"name": "X_train", "data": X_train}])
    # normalizer.fit("StandardScale", feature_columns)
    # # norm.fit(settings["normalization_fit_type"], settings["features_name"], settings["normalization_fit_target"])]

    # # Get normalized data
    # X_train_scaled = normalizer.get_normalized_data(0)  # Get the scaled data
    # X_test_scaled = normalizer.get_normalized_data(1)  # Get scaled test data
    # X_val_scaled = normalizer.get_normalized_data(2)  # Get scaled validation data

    # Prepare the features dictionary
    features_dict = {
        "df": original_df,
        "processed_df": df,
        "feature_columns": feature_columns, 
        # "X_train": X_train, 
        # "X_test": X_test, 
        # "y_train_reg": y_train_reg, 
        # "y_train_cla": y_train_cla,
        # "X_train_scaled": X_train_scaled,
        # "split_train_test": split_train_test,
        # "split_test_valid": split_test_valid,
        # "X_test_scaled": X_test_scaled,
        # "y_test_reg": y_test_reg,
        # "X_val": X_val,
        # "X_val_scaled": X_val_scaled,
        # "y_val_reg": y_val_reg,
    }

    # # Apply PCA if specified
    # if compress_features:
    #     pca = PCA(n_components=3)
    #     X_train_pca = pca.fit_transform(X_train_scaled)
    #     X_test_pca = pca.transform(X_test_scaled)
    #     X_val_pca = pca.transform(X_val_scaled)

    #     features_dict["X_train_pca"] = X_train_pca
    #     features_dict["X_test_pca"] = X_test_pca
    #     features_dict["X_val_pca"] = X_val_pca

    # Return all relevant datasets as a dictionary for clarity and organization
    return features_dict


## Apply Deep Learning Algorithms

In [ ]:
final_assets["Symbol"].to_list()

### Settings

The "horizon" in the settings refers to how many steps into the future the model is tasked with predicting, using the current and past data points. In your case:

- **Horizon (int)**: This specifies the number of time steps or terms that the model should predict into the future.
  
  For example, if you set the horizon to `120`, this means the model will attempt to predict the next 120 future time steps based on the historical data it is trained on. 

- **Used Time Frame**: In this case, we are using a 1-minute time frame, which means that each time step represents 1 minute of data.

- **Total Future Time Covered**: The horizon multiplied by the time frame gives you the total future time that the model is expected to see or predict.
  
  - For example, with a 1-minute time frame and a horizon of 120, the model is expected to predict the next 120 minutes (i.e., 2 hours) into the future.

**Formula:**

**Total future time = horizon X used time frame**

In this case, using the 15-minute time frame:
- With a **8 horizon**, the total prediction span will be:
  **8 X 15 min = 120 minutes (2 hours)**

Thus, the model is expected to predict up to 2 hours into the future using a 120-minute horizon with 15-minute granularity.

Mins in a day = 24 * 60 * 60 = 93600


In [35]:
settings = {
                 "target_markets": ['Volatility 25 Index', 'Volatility 75 (1s) Index', 'Boom 500 Index', 'Step Index'],
                 "normalization_fit_type": "StandardScale",
                 "normalization_fit_target": "all",
                 "normalization_target": "close",
                 "label_disparity": 0.095, # 0.025, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.1
                 "features_name": ['open', 'high', 'low', 'close', 'volume'], # ["ask", "bid", "high", "low", "moy_prev_day"], 
                 "labels_name": ["buy", "sell", "neutral"], # ["buy", "sell", "neutral"] | ["buy", "sell", "wait"] | ["strong buy", "buy", "strong sell", "sell", "neutral"]
                 "is_balance": True,
                 "split_train": 0.8,
                 "split_test": 0.2,
                 "horizon": 8, 
                 "delta_neurons_numbers": 4, # delta {2-10}
                 "model_optimizer": "adam",
                 "model_loss": "CrossEntropyLoss", 
                 "model_epoch": 120, # 50, 70, 100, 120, 150, 250
                 "model_batch_size": 32, # 50, 32, 25
                 "model_validation_split": 0.15,
                 "model_shuffle": False
                }

TIME_FRAME = mt5.TIMEFRAME_M15
LABEL_TARGET: str = settings["normalization_target"]

In [ ]:
from tqdm import tqdm
import concurrent.futures

# Initialize MetaTrader 5 or other data source
mt5.initialize()

# Symbols list from final_assets
dl_selected_symbols = settings['target_markets']

dl_selected_symbols

In [37]:
# # Conversion factors
# years_to_days = 365.25  # Average days in a year accounting for leap years
# days_to_hours = 24  # Hours in a day
# hours_to_minutes = 60  # Minutes in an hour

# # Converting years to minutes
# years = 3
# minutes = years * years_to_days * days_to_hours * hours_to_minutes
# minutes

### Fetch Data with All Available TA Indicators 

From January 2022 to January 2024

In [ ]:
# To store results
processed_data_results = []

def process_symbol(symbol):
    try:
        print(f"Processing symbol: {symbol}")
        # Retrieve data for the symbol
        df = get_data(symbol, 0, timeframe=TIME_FRAME, is_range=True, 
                      utc_from=datetime(2022, 1, 1), utc_to=datetime(2024, 1, 1)).dropna()
        # print(df)
        
        # Perform feature engineering
        processed_data = features_engineering(df, add_ta=True)
        settings['features_name'] = processed_data['feature_columns']
        return processed_data
    
    except Exception as e:
        print(f"Issue during data importation or processing for symbol {symbol}: {e}")
        return None  # Return None in case of an error

# Use ThreadPoolExecutor or ProcessPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Use tqdm to show progress
    futures = {executor.submit(process_symbol, symbol): symbol for symbol in dl_selected_symbols}

    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
        result = future.result()
        if result is not None:
            processed_data_results.append(result)


In [ ]:
len(processed_data_results)

In [ ]:
processed_data_results[0]['processed_df']


In [ ]:

# Display
for i, name in enumerate(dl_selected_symbols):
    fig = plt.figure(figsize=(21, 7))

    # Plot the target label's data
    processed_data_results[i]['processed_df'][LABEL_TARGET].plot(label=LABEL_TARGET, title=f"{LABEL_TARGET} Price Curve".capitalize())

    # Add a watermark or name to the background
    plt.text(
    0.5, 0.5,                    # X and Y position (centered in the plot)
    name,                          # The text to display
    fontsize=50,                  # Font size for visibility
    color='gray',                 # Color of the watermark
    alpha=0.3,                    # Transparency level
    ha='center',                  # Horizontal alignment
    va='center',                  # Vertical alignment
    transform=plt.gca().transAxes  # Ensure the text is relative to the plot's axes
    )

    # Display grid and legend
    plt.grid()
    plt.legend()
    plt.show()


In [ ]:
stop

### Apply Nomalization

In [ ]:
settings["features_name"] = processed_data_results[0]['processed_df'].columns

settings["features_name"]

In [54]:
ndata = []
for i, name in enumerate(dl_selected_symbols):
    ndata.append({"name": name, "data": processed_data_results[i]['processed_df']})

In [ ]:
# NORMALIZATION OF THE DATA
norm = Normalizer(data=ndata)
norm.fit(settings["normalization_fit_type"], settings["features_name"], settings["normalization_fit_target"])


In [57]:
normalize_data_list = []
for i in range(len(dl_selected_symbols)):
    normalize_data_list.append(norm.get_normalized_data(i))


In [ ]:
normalize_data_list[0]

In [ ]:
normalize_data_list[1]

### Apply PEAKS DETECTION

In [61]:
def estimate_peak_distance(peaks_max, peaks_min):
    """
    Estimate peak distance between maximum and minimum peaks.

    Args:
        peaks_max (np.ndarray): Array of maximum peak values.
        peaks_min (np.ndarray): Array of minimum peak values.

    Returns:
        dict: A dictionary containing the mean and distance values for both peaks_max and peaks_min.
    """
    # Flatten the peak arrays
    peaks_max_flat = peaks_max.flatten()
    peaks_min_flat = peaks_min.flatten()

    # Calculate mean values
    mean_max = np.mean(peaks_max_flat)
    mean_min = np.mean(peaks_min_flat)

    # Calculate the difference between max peak and mean
    distance_max = np.max(peaks_max_flat) - mean_max
    distance_min = np.max(peaks_min_flat) - mean_min

    # Return results as a dictionary
    return {
        'mean_max': mean_max,
        'mean_min': mean_min,
        'distance_max': distance_max,
        'distance_min': distance_min
    }


In [62]:
def detect_peaks_for_all(normalize_data_list, settings):
    """
    Dynamically detect peaks (max and min) for each dataset in the list.

    Args:
        normalize_data_list (list): List of normalized data arrays.
        settings (dict): Settings containing normalization target and label disparity.

    Returns:
        list: A list of tuples containing (peaks_max, peaks_min) for each dataset.
    """
    peaks_results = []

    for i, normalize_data in enumerate(normalize_data_list):
        # Apply peaks detection for each dataset in the list
        peaks_max, peaks_min = peaks_detection(normalize_data[settings["normalization_target"]], settings["label_disparity"])
        
        # Append the results as a tuple (peaks_max, peaks_min)
        peaks_results.append((peaks_max, peaks_min))

    return peaks_results


In [ ]:
# peaks_max, peaks_min = peaks_detection(normalize_data[settings["normalization_target"]], settings["label_disparity"])
peaks_results = detect_peaks_for_all(normalize_data_list, settings)

peaks_results

In [ ]:
for i, peaks_result in enumerate(peaks_results):
    peak_max, peak_min = peaks_results
    print(f"{i} => ", estimate_peak_distance(peak_max, peak_min))


In [ ]:
# Display peaks
fig = plt.figure(figsize=(21, 7))
plt.plot(peaks_max[:,0], peaks_max[:, 1], 'ro', label="Maximum peaks")
plt.plot(peaks_min[:,0], peaks_min[:, 1], 'go', label="Minimum valleys")
plt.plot(normalize_data[LABEL_TARGET], label=LABEL_TARGET.capitalize())
plt.grid()
plt.title("Peaks detection")
plt.legend()
plt.show()

### Apply Frame labelization

LABELIZATION WITH PEAKS

In [ ]:
%%time

labeler = MarketLabeler(peaks_max, peaks_min)
labeled_normalize_df = labeler.label_dataframe(normalize_data, LABEL_TARGET)

In [ ]:
labeled_normalize_df

### Apply Data Generator - Split Normalized Data

horizon (int): The number of forward terms of the target time series to be estimated

In [62]:
data_generator = DataGenerator(labeled_normalize_df, settings["horizon"], settings["features_name"], settings["labels_name"])


In [ ]:
%%time
x_train, y_train, x_test, y_test = data_generator.train_test_split(settings["split_test"], settings["is_balance"])


In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_train.dtype, y_train.dtype


In [ ]:
y_train[0]

### Model Experiments

In [67]:
# from keras.layers import Input, Dense, Dropout, Bidirectional, Reshape
# from keras.callbacks import EarlyStopping
# from keras.metrics import F1Score, FBetaScore, Precision, Recall 
# from tcn import TCN, tcn_full_summary

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import Adam
from pytorch_tcn import TCN

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score

# https://github.com/paul-krug/pytorch-tcn
# https://github.com/locuslab/TCN/

#### Define Model Util functions (for Neural Nets)

In [68]:
def calculate_lstm_neurons(data_array: np.array, settings: dict) -> int:
    """
    Calculates the number of neurons for an LSTM network based on data and settings.

    Args:
        data_array (np.array): The numpy array containing the training data.
        settings (dict): A dictionary containing settings for neuron calculation.
            - features_name (list): Names of features for training.
            - labels_name (list): Names of output labels (classes).
            - delta_neurons_numbers (int): Hyperparameter scaling the number of neurons.

    Returns:
        int: The calculated number of neurons for the LSTM network.

    Raises:
        ValueError: If required keys are missing from the settings dictionary.
    """

    if not all(key in settings for key in ["features_name", "labels_name", "delta_neurons_numbers"]):
        raise ValueError("Missing required keys in 'settings' dictionary.")

    num_features = len(settings["features_name"])
    num_labels = len(settings["labels_name"])
    num_samples = data_array.shape[0]
    scaling_factor = settings["delta_neurons_numbers"]

    return int(num_samples / (scaling_factor * (num_features + num_labels)))


In [69]:
from torch.utils.data import TensorDataset, DataLoader

def create_train_test_set(x_train, x_test, y_train, y_test):
    """Creates train and test datasets from numpy arrays or lists.

    Args:
        x_train: Training features.
        x_test: Test features.
        y_train: Training labels.
        y_test: Test labels.

    Returns:
        training_data: A TensorDataset for the training set.
        test_data: A TensorDataset for the test set.
    """
    # Convert to torch tensors
    tensor_x_train = torch.Tensor(x_train)
    tensor_y_train = torch.Tensor(y_train)
    training_data = TensorDataset(tensor_x_train, tensor_y_train)  # Create training dataset

    tensor_x_test = torch.Tensor(x_test)
    tensor_y_test = torch.Tensor(y_test)
    test_data = TensorDataset(tensor_x_test, tensor_y_test)  # Create test dataset

    return training_data, test_data


In [ ]:
# https://stackoverflow.com/questions/40761185/what-is-the-intuition-of-using-tanh-in-lstm
# x_tcn = GlobalAveragePooling1D()(x_tcn)
 
# SET NUMBER OF NEURONS
settings["model_neural"] = calculate_lstm_neurons(x_train, settings)  
settings["model_neural_hidden"] =  settings["model_neural"] + int(settings["model_neural"] / 2) # int(settings["model_neural"] / 2)

settings["model_neural"], settings["model_neural_hidden"]

In [71]:
# Model parameters
input_dim = x_train.shape[2]  # Feature size per time step (input dimension)
hidden_dim_1 = settings["model_neural"]  # Hidden layer size
hidden_dim_2 = settings["model_neural_hidden"]  # Hidden layer size
output_dim = y_train.shape[1]  # Number of output classes
device = "cpu"
models = {}
trained_models = {}

In [72]:
training_data, test_data = create_train_test_set(x_train, x_test, y_train, y_test)

train_dataloader = DataLoader(training_data, batch_size=settings["model_batch_size"], shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=settings["model_batch_size"], shuffle=False)

#### Model A (Bi-LSTM)

In [ ]:
class BidirectionalLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers: int = 4, **kwargs):
        super(BidirectionalLSTMModel, self).__init__()

        self.hidden_size = hidden_size or 24

        self.num_layers = num_layers

        # Bidirectional LSTMs
        self.bilstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True, dropout=0.2)

        # Dropout layer
        # self.dropout = nn.Dropout(0.2)

        # Dense (fully connected) layer with softmax activation
        self.fc = nn.Linear(hidden_size*2, output_size) 

    def forward(self, x):
        # Set initial hidden and cell states 
        # h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        # c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)

        # Passing input through LSTMs
        out, _ = self.bilstm(x) # out, hidden = self.bilstm(x, (h0, c0))
        # out = self.dropout(out)

        # Taking the output of the last time step
        out = out[:, -1, :]

        fc = self.fc(out)
        x = F.softmax(fc)
        return x


# Model instantiation
models["BidirectionalLSTM"] = BidirectionalLSTMModel(input_size=input_dim, hidden_size=hidden_dim_1, output_size=output_dim)

# Summary 
print(models["BidirectionalLSTM"])


#### Model B (Bi-GRU)

In [ ]:
class BidirectionalGRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers: int = 4, **kwargs):
        super(BidirectionalGRUModel, self).__init__()

        self.hidden_size = hidden_size or 24

        self.num_layers = num_layers

        # Bidirectional GRU layers
        self.bigru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True, dropout=0.2)
        
        # Dense (fully connected) layer with softmax activation
        self.fc = nn.Linear(hidden_size*2, output_size) 
        
    def forward(self, x):
        # Forward pass through bidirectional GRU layers
        out, _ = self.bigru(x)
        
        # Taking the output of the last time step
        out = out[:, -1, :]

        fc = self.fc(out)
        x = F.softmax(fc)
        return x

models["BidirectionalGRU"] = BidirectionalGRUModel(input_size=input_dim, hidden_size=hidden_dim_1, output_size=output_dim)

# Summary 
print(models["BidirectionalGRU"])

#### Model C (TCN)

In [ ]:
class TCNModel(nn.Module):
    def __init__(self, input_size, output_size, num_channels: Tuple = (2, 3, 5, 8, 16, 32), kernel_size: int = 5, dilations: Tuple = (1, 2, 4, 8, 16, 32)):
        super(TCNModel, self).__init__()

        # Define the TCN layer with appropriate parameters
        self.tcn = TCN(input_size, num_channels, kernel_size, dilations, input_shape='NLC', dropout=0.2)

        # Fully connected layer with input size matching the last TCN channel size
        self.fc = nn.Linear(num_channels[-1], output_size)

    def forward(self, x):
        # Passing input through the TCN layer
        out = self.tcn(x)

        # Taking the output of the last time step
        out = out[:, -1, :]

        # Pass through the fully connected layer
        fc = self.fc(out)
        
        x = F.softmax(fc)
        return x  


# Model instantiation
models["TCN"] = TCNModel(input_size=input_dim, output_size=output_dim)

# Summary 
print(models["TCN"])


In [ ]:
models

#### Training


In [77]:
def train_model(model: nn.Module,
                model_optimizer: str,
                model_loss: str,
                num_epochs: int = 20,
                dataloader: Optional[DataLoader] = None,
                device: Optional[str] = None) -> tuple[nn.Module, optim.Optimizer, nn.Module]:
    """Trains a PyTorch model.

    Args:
        model: The PyTorch model to train.
        model_optimizer: The name of the optimizer to use.
        model_loss: The name of the loss function to use.
        num_epochs: The number of epochs to train for.
        dataloader: The DataLoader object for loading data.
        device: The device to use for training (e.g., 'cpu', 'cuda').

    Returns:
        A tuple containing the trained model, optimizer, and loss function.
    """

    if dataloader is None:
        raise ValueError("A DataLoader object must be provided.")

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)

    # Optimizer
    optimizer_class = getattr(optim, model_optimizer.capitalize())
    optimizer = optimizer_class(model.parameters())

    # Loss function
    loss_function = getattr(nn, model_loss)()

    for epoch in range(num_epochs):
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, targets)
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

    return model, optimizer, loss_function

In [ ]:
for name, model in models.items(): 
    print(f"{name} Model:")
    trained_models[name] = train_model(models[name], settings["model_optimizer"], settings["model_loss"], settings['model_epoch'], train_dataloader)


In [79]:
# # Plot real-time
# plotRT = PlotRealTime()

# # Learning stats
# def lr(epoch):
#     if epoch < 15:
#         return 0.0001 # 0.010
#     if epoch < 40:
#         return 0.0095
#     if epoch < 55:
#         return 0.008
#     if epoch < 60:
#         return 0.007
#     if epoch < 150:
#         return 0.0001 # 0.01
    
#     return 0.0001

# def lr2(epoch):
#     if epoch < 100:
#         return 1e-8
    
#     return 1e-4


# # Define early stopping callback
# patience = 25
# early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)


# plotter = PlotRealTime()
# history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), callbacks=[plotter])


# %%time
# hist_a = model_a.fit(x_train,
#                  y_train, 
#                  epochs=settings["model_epoch"],
#                  batch_size=settings["model_batch_size"],
#                  validation_split=settings["model_validation_split"],
#                  verbose=1,
#                  shuffle=settings["model_shuffle"],
#                  callbacks=[plotRT, early_stopping, LearningRateScheduler(lr, verbose=1)])

# model_a.save(f"../models/{file_name}_model_a_{time.time()}.h5")

#### Evaluate

np.argmax(y_test, axis=1) => {0: 'BUY', '1': 'SELL', 2: 'NEUTRAL'}

In [80]:
metrics = {}

In [81]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from torch.optim import Optimizer
from torch.utils.data import DataLoader
from typing import Tuple, Optional

def evaluate_model(trained_model: Tuple[nn.Module, Optimizer, nn.Module],
                   dataloader: DataLoader,
                   device: Optional[str] = None) -> dict:
    """Evaluates a PyTorch model on a given dataset.

    Args:
        trained_model: A tuple containing the trained PyTorch model, optimizer, and loss function.
        dataloader: The DataLoader object for loading evaluation data.
        device: The device to use for evaluation (e.g., 'cpu', 'cuda').

    Returns:
        A dictionary containing evaluation metrics like accuracy, precision, recall, F1 score, and confusion matrix.
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model, optimizer, loss_function = trained_model
    model.to(device)
    model.eval()  # Set the model to evaluation mode


    raw_targets = []
    raw_predictions = []
    all_targets = []
    all_predictions = []
    total_loss = 0

    # Loss function for evaluation 
    criterion = loss_function

    with torch.no_grad():  # Disable gradient calculation during evaluation
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()

            raw_targets.extend(targets.cpu().numpy())
            raw_predictions.extend(outputs.cpu().numpy())

            # Convert outputs to predicted class (taking the argmax for multi-class classification)
            _, predicted = torch.max(outputs, 1)

            # If targets are one-hot encoded, convert them to class indices
            if len(targets.shape) > 1 and targets.shape[1] > 1:
                targets = torch.argmax(targets, dim=1)

            all_targets.extend(targets.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    # Calculate evaluation metrics
    accuracy = accuracy_score(all_targets, all_predictions)
    precision = precision_score(all_targets, all_predictions, average='weighted')
    recall = recall_score(all_targets, all_predictions, average='weighted')
    f1 = f1_score(all_targets, all_predictions, average='weighted')

    # Confusion matrix
    conf_matrix = confusion_matrix(all_targets, all_predictions)

    # Return the evaluation metrics in a dictionary
    metrics = {
        "loss": total_loss / len(dataloader),
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix,
        "raw_targets": raw_targets,
        "raw_predictions": raw_predictions,
    }

    # Display the metrics
    print(f"Evaluation Loss: {metrics['loss']:.4f}")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1_score']:.4f}")
    print(f"Confusion Matrix:\n{metrics['confusion_matrix']}")

    return metrics


In [ ]:
for name, model in trained_models.items():
    print(f"{name} Model:")
    metrics[name] = evaluate_model(trained_models[name], test_dataloader)
    

##### Performance Table

In [ ]:
dl_results = pd.DataFrame(metrics).T.drop(columns=['raw_targets', 'raw_predictions'])

dl_results

##### Confusion Matrix

In [84]:
import seaborn as sns

def plot_confusion_matrix(conf_matrix, label_map: dict = {0: 'BUY', 1: 'SELL', 2: 'NEUTRAL'}):
    """Plots the confusion matrix using a heatmap.

    Args:
        conf_matrix: The confusion matrix to plot (numpy array or similar).
        label_map: A dictionary mapping label indices to actual class names.

    Returns:
        None. Displays the confusion matrix plot.
    """
    plt.figure(figsize=(10, 8))
    ax = plt.subplot()

    # Create heatmap with the confusion matrix
    sns.heatmap(conf_matrix, annot=True, fmt='g', ax=ax)  # annot=True to annotate cells, fmt='g' for plain numbers

    # Add labels, title, and ticks
    ax.set_xlabel('Predicted Labels')
    ax.set_ylabel('Actual Labels')
    ax.set_title('Confusion Matrix')

    # Set tick labels based on label_map
    labels = list(label_map.values())
    ax.xaxis.set_ticklabels(labels)
    ax.yaxis.set_ticklabels(labels)

    # Show the plot
    plt.show()


In [ ]:
for name, metric in metrics.items():
    print(f"{name} Model:")
    conf_matrix = metric['confusion_matrix']
    plot_confusion_matrix(conf_matrix, label_map={0: 'BUY', 1: 'SELL', 2: 'NEUTRAL'})
    

##### ROC curve

In [86]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

def plot_roc_curve(y_pred, y_test, classes=[0, 1, 2]):
    """
    Plots the ROC curves for multi-class classification.

    Args:
        y_pred (numpy array): Predicted probabilities for each class, shape (n_samples, n_classes).
        y_test (numpy array): True class labels, shape (n_samples,).
        classes (list): List of classes. Default is [0, 1, 2].

    Returns:
        None. Displays the ROC curve plot.
    """
    # Binarize the output for multi-class ROC
    y_test_bin = label_binarize(y_test, classes=classes)
    n_classes = len(classes)

    # Initialize dictionaries to store False Positive Rates, True Positive Rates, and AUCs for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    # Compute ROC curve and AUC for each class
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and AUC
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_pred.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # Compute macro-average ROC curve and AUC
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
    mean_tpr = np.zeros_like(all_fpr)

    # Interpolate all ROC curves and compute the mean True Positive Rate (macro-average)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot the ROC curves
    plt.figure(figsize=(10, 8))

    # Micro-average ROC curve
    plt.plot(fpr["micro"], tpr["micro"],
             label=f'Micro-average ROC curve (area = {roc_auc["micro"]:0.2f})',
             color='blue', linestyle=':', linewidth=4)

    # Macro-average ROC curve
    plt.plot(fpr["macro"], tpr["macro"],
             label=f'Macro-average ROC curve (area = {roc_auc["macro"]:0.2f})',
             color='navy', linestyle=':', linewidth=4)

    # ROC curves for each class
    colors = ['aqua', 'darkorange', 'cornflowerblue']
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label=f'ROC curve of class {classes[i]} (area = {roc_auc[i]:0.2f})')

    # Plot the diagonal (random classifier)
    plt.plot([0, 1], [0, 1], 'k--', lw=2)

    # Add labels, title, and legend
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.show()


In [ ]:
for name, metric in metrics.items():
    print(f"{name} Model:")

    # Convert lists of arrays to 2D NumPy arrays
    raw_predictions = np.array(metric['raw_predictions'])
    raw_targets = np.array(metric['raw_targets'])

    plot_roc_curve(raw_predictions, raw_targets)
    

#### Save Models

In [88]:
from pathlib import Path

model_dir = Path(os.getcwd()).parent / 'models'
# model_dir = os.path.join(os.getcwd(), 'models')
# os.makedirs(model_dir, exist_ok=True)  # Create directory if it doesn't exist


def save_model(trained_model: Tuple[nn.Module, optim.Optimizer, nn.Module], path: str) -> None:
    """
    Saves the complete training state of a PyTorch model to a file.

    This function saves the model's state dictionary, containing the learned
    parameters (weights and biases), along with the state dictionaries of the
    optimizer and the loss function. This allows you to load the entire training
    state later and resume training from the saved point or use the model for
    inference.

    Args:
        trained_model: A tuple containing the trained PyTorch model (nn.Module),
                       optimizer (optim.Optimizer), and loss function (nn.Module).
        path: The path to the file where the training state will be saved (str).

    Returns:
        None
    """

    model, optimizer, loss_function = trained_model
    state_dict = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss_function': loss_function.state_dict(),  # Save loss function instance (for class-based)
    }
    torch.save(state_dict, path)


def load_model(model_definitions: Tuple[nn.Module, optim.Optimizer, nn.Module], path: str) -> Tuple[nn.Module, optim.Optimizer, nn.Module]:
    """
    Loads a previously saved PyTorch model training state from a file.

    Args:
        path: The path to the file where the training state is saved (str).

    Returns:
        A tuple containing the loaded PyTorch model (nn.Module), optimizer (optim.Optimizer),
        and loss function (nn.Module).
    """
    if not os.path.isfile(path):
        raise FileNotFoundError(f"No model file found at {path}.")

    model, optimizer, loss_function = model_definitions
    state_dict = torch.load(path)
    model = model.load_state_dict(state_dict['model_state_dict'])
    optimizer = optimizer.load_state_dict(state_dict['optimizer_state_dict'])
    loss_function = loss_function.load_state_dict(state_dict['loss_function']) 

    return model, optimizer, loss_function

In [ ]:
for name, model in trained_models.items():
    print(f"{name} Model:")
    print(model)
    # Save the model using joblib
    model_filename = os.path.join(model_dir, f"{dl_selected_symbols.iloc[0]}_{name}.pth")
    save_model(model, model_filename)
    print(f"Model saved to {model_filename}")


In [ ]:
# Load Models
model_definitions = trained_models['BidirectionalLSTM']
model_filename = os.path.join(model_dir, f"{dl_selected_symbols.iloc[0]}_BidirectionalLSTM.pth")

trained_model = load_model(model_definitions, model_filename)
trained_model

# DUMP

In [ ]:
# # Import the necessary libraries
# from sklearn.svm import SVC
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.linear_model import LogisticRegression
# from tqdm import tqdm

# # Initialize models
# tree = DecisionTreeClassifier(max_depth=20)
# svr = SVC(C=1.5)
# lin = LogisticRegression()

# # Initialize MetaTrader 5 or other data source
# mt5.initialize()

# # Symbols list from final_assets
# symbols = final_assets["Symbol"]

# # To store results
# results_list = []

# # Iterate over all symbols
# for symbol in tqdm(symbols):
#     print(symbol, "\n")
#     try:
#         # Retrieve data for the symbol
#         df = get_data(symbol, 3500, mt5.TIMEFRAME_D1).dropna()
        
#         # Perform feature engineering
#         processed_data = features_engineering(df)

#         # Decision Tree
#         sharpe_tree = predictor(processed_data, tree, reg=True)
#         results_list.append([symbol, "Tree", sharpe_tree, len(df)])

#         # # SVM (Support Vector Classifier)
#         sharpe_svr = predictor(processed_data, svr, reg=False)
#         results_list.append([symbol, "SVR", sharpe_svr, len(df)])

#         # Logistic Regression
#         sharpe_linreg = predictor(processed_data, lin, reg=False)
#         results_list.append([symbol, "LinReg", sharpe_linreg, len(df)])

#     except KeyError:
#         continue
    
#     except Exception as e:
#         print(f"Issue during data importation or processing for symbol {symbol}: {e}")
#         raise



In [91]:
# from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer as SklearnNormalizer

# class Normalizer:
#     """
#     A class for normalizing market data while preserving the original data.

#     This class applies logarithm and a normalization (MinMax, StandardScale, Normalizer_l1, Normalizer_l2).
#     """

#     def __init__(self, data):
#         """
#         Initializes the Normalizer class with market data.

#         Args:
#             data (list[dict]): A list of dictionaries, where each dictionary represents a market
#                                with keys like "name" and "data" (containing market data).
#         """
#         self.__market_names = [e["name"] for e in data]
#         self.__original_data = [pd.DataFrame(e["data"]) for e in data]  # Convert to DataFrame
#         self.__normalized_data = None  # Stores normalized data after fit is called

#     def __normalize(self, target, numerical_data, scaler_func):
#         """
#         Normalizes data using the provided scaler function for specific target markets.

#         Args:
#             target (str): The target market(s) to normalize ("all" or a list of market names).
#             numerical_data (list[pd.DataFrame]): A list of DataFrames containing numerical data for each market.
#             scaler_func (callable): A function that performs data scaling (e.g., MinMaxScaler.fit_transform).

#         Returns:
#             list[pd.DataFrame]: List of normalized market DataFrames.
#         """
#         normalized_markets = []
#         target_markets = self.__market_names if target == "all" else target

#         for market_name, market in zip(self.__market_names, numerical_data):
#             if market_name in target_markets:
#                 columns = market.columns
#                 market_scaled = scaler_func(market)
#                 normalized_markets.append(pd.DataFrame(market_scaled, columns=columns))

#         return normalized_markets

#     def fit(self, norm_type, features_list, target="all"):
#         """
#         Performs data normalization based on the specified type, features, and target market(s).

#         Args:
#             norm_type (str): The type of normalization to perform ("MinMax", "StandardScale", "Normalizer_l1", or "Normalizer_l2").
#             features_list (list): A list of feature names to consider for normalization.
#             target (str, optional): The target market(s) to normalize ("all" or a list of market names). Defaults to "all".

#         Raises:
#             ValueError: If an invalid normalization type is provided.
#         """
#         # Extract numerical data for the specified features
#         numerical_data = [market[features_list]._get_numeric_data() for market in self.__original_data]

#         if norm_type == "MinMax":
#             print(f"Performing MinMax Normalization on {target}.")
#             scaler_func = MinMaxScaler().fit_transform
#         elif norm_type == "StandardScale":
#             print(f"Performing StandardScale Normalization on {target}.")
#             scaler_func = StandardScaler().fit_transform
#         elif norm_type.startswith("Normalizer"):
#             norm_value = norm_type.split("_")[-1]  # Extract l1 or l2 from "Normalizer_l1" or "Normalizer_l2"
#             if norm_value not in ("l1", "l2"):
#                 raise ValueError("Invalid norm type for Normalizer. Must be 'l1' or 'l2'.")
#             print(f"Performing Normalizer (norm={norm_value}) on {target}.")
#             scaler_func = SklearnNormalizer(norm=norm_value).fit_transform
#         else:
#             raise ValueError(f"Invalid normalization type: {norm_type}")

#         self.__normalized_data = self.__normalize(target, numerical_data, scaler_func)

#     def get_normalized_data(self, idx: int = None):
#         """
#         Returns the normalized data if normalization has been performed, otherwise raises an error.
        
#         Args:
#             idx (int, optional): The location/index of the normalized data. If None, returns all normalized data.

#         Raises:
#             RuntimeError: If data has not been normalized yet.
#             IndexError: If the index is out of range.
#         """
#         if self.__normalized_data is None:
#             raise RuntimeError("Data has not been normalized yet. Please call 'fit' first.")
        
#         if idx is not None:
#             if idx < 0 or idx >= len(self.__normalized_data):
#                 raise IndexError("Index out of range.")
#             return self.__normalized_data[idx]
        
#         return self.__normalized_data  # Return all normalized data if idx is None

#     def get_original_data(self):
#         """
#         Returns the original, un-normalized data.
#         """
#         return [data.copy() for data in self.__original_data]  # Return a deep copy


In [92]:
# from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer as SklearnNormalizer

# class Normalizer:
#     """
#         A class for normalizing market data while preserving the original data.
#     """

#     def __init__(self, data):
#         """
#             Initializes the Normalizer class with market data.

#             Args:
#                 data (list[dict]): A list of dictionaries, where each dictionary represents a market
#                                    with keys like "name" and "data" (containing market data).
#         """
#         self.__market_names = [e["name"] for e in data]
#         self.__original_data = [e["data"] for e in data]  # TODO: Deep copy to preserve original data
#         self.__normalized_data = None  # Stores normalized data after fit is called

#     def __normalize(self, target, numerical_data, scaler_func):
#         """
#             Normalizes data using the provided scaler function for specific target markets.

#             Args:
#                 target (str): The target market(s) to normalize ("all" or a list of market names).
#                 numerical_data (list[pd.DataFrame]): A list of DataFrames containing numerical data for each market.
#                 scaler_func (callable): A function that performs data scaling (e.g., MinMaxScaler.fit_transform).
#         """
#         normalized_markets = []
#         if target == "all":
#             for market in numerical_data:
#                 columns = market.columns
#                 market_scaled = scaler_func(market)
#                 market_scaled = pd.DataFrame(market_scaled, columns=columns)
#                 normalized_markets.append(market_scaled)
#         else:
#             for market_name, market in zip(self.__market_names, numerical_data):
#                 if market_name in target:
#                     columns = market.columns
#                     market_scaled = scaler_func(market)
#                     market_scaled = pd.DataFrame(market_scaled, columns=columns)
#                     normalized_markets.append(market_scaled)
                    
#         return normalized_markets

#     def fit(self, norm_type, features_list, target="all"):
#         """
#             Performs data normalization based on the specified type, features, and target market(s).

#             Args:
#                 norm_type (str): The type of normalization to perform ("MinMax", "StandardScale", "Normalizer_l1", or "Normalizer_l2").
#                 features_list (list): A list of feature names to consider for normalization.
#                 target (str, optional): The target market(s) to normalize ("all" or a list of market names). Defaults to "all".

#             Raises:
#                 ValueError: If an invalid normalization type is provided.
#         """
#         # Extract numerical data for the specified features
#         numerical_data = [market[features_list]._get_numeric_data() for market in self.__original_data]

#         if norm_type == "MinMax":
#             print(f"Performing MinMax Normalization on {target}.")
#             scaler_func = MinMaxScaler().fit_transform
#             self.__normalized_data = self.__normalize(target, numerical_data, scaler_func)
#         elif norm_type == "StandardScale":
#             print(f"Performing StandardScale Normalization on {target}.")
#             scaler_func = StandardScaler().fit_transform
#             self.__normalized_data = self.__normalize(target, numerical_data, scaler_func)
#         elif norm_type.startswith("Normalizer"):
#             norm_value = norm_type.split("_")[-1]  # Extract l1 or l2 from "Normalizer_l1" or "Normalizer_l2"
#             if norm_value not in ("l1", "l2"):
#                 raise ValueError("Invalid norm type for Normalizer. Must be 'l1' or 'l2'.")
#             print(f"Performing Normalizer (norm={norm_value}) on {target}.")
#             scaler_func = Normalizer(norm=norm_value).fit_transform
#             self.__normalized_data = self.__normalize(target, numerical_data, scaler_func)
#         else:
#             raise ValueError(f"Invalid normalization type: {norm_type}")

#     def get_normalized_data(self, idx: int = 0):
#         """
#             Returns the normalized data if normalization has been performed, otherwise raises an error.
            
#             Args:
#                 idx (int): The location/index of the normalized data.

#             Raises:
#                 RuntimeError: If data has not been normalized yet.
#                 IndexError: If the index is out of range.
#         """
#         if self.__normalized_data is None:
#             raise RuntimeError("Data has not been normalized yet. Please call 'fit' first.")
        
#         if idx < 0 or idx >= len(self.__normalized_data):
#             raise IndexError("Index out of range.")
        
#         if idx is None:
#             return self.__normalized_data
        
#         return self.__normalized_data[idx]

#     def get_original_data(self):
#         """
#             Returns the original, un-normalized data.
#         """
#         return self.__original_data.copy


In [93]:
# def train_model(model, model_optimizer: str, model_loss: str, num_epochs: int = 20, dataloader: Optional[DataLoader] = None):
#     """ 
#     Train Model
#     """
#     # Optimizer
#     optimizer_class = getattr(optim, model_optimizer.capitalize())
#     optimizer = optimizer_class(model.parameters())

#     # Loss function
#     loss_function = getattr(nn, model_loss)()   # Use CrossEntropyLoss for multi-class classification

#     for epoch in range(num_epochs):
#         for inputs, targets in dataloader:  # use DataLoader for batch loading
#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = loss_function(outputs, targets)
#             loss.backward()
#             optimizer.step()

#     print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

#     return model, optimizer, loss_function

In [94]:
# def create_train_test_set(x_train, x_test, y_train, y_test):
#     """Creates train and test datasets from numpy arrays or lists.

#     Args:
#         x_train: Training features.
#         x_test: Test features.
#         y_train: One-hot encoded training labels.
#         y_test: One-hot encoded test labels.

#     Returns:
#         training_data: A TensorDataset for the training set.
#         test_data: A TensorDataset for the test set.
#     """
#     # Convert to torch tensors
#     tensor_x_train = torch.Tensor(x_train)
#     tensor_y_train = torch.argmax(torch.Tensor(y_train), dim=1).long()  # Convert one-hot to class indices
#     training_data = TensorDataset(tensor_x_train, tensor_y_train)  # Create training dataset

#     tensor_x_test = torch.Tensor(x_test)
#     tensor_y_test = torch.argmax(torch.Tensor(y_test), dim=1).long()  # Convert one-hot to class indices
#     test_data = TensorDataset(tensor_x_test, tensor_y_test)  # Create test dataset

#     return training_data, test_data


In [95]:

# def predictor(data: dict, model, reg: bool = True, spread: float = 0.035, compressed_features: bool = True) -> float:
#     """
#     Fits the model to the training data, makes predictions on the entire dataset, 
#     and computes the strategy's Sharpe ratio based on the predictions.
    
#     Args:
#         data (dict): The output of the features_engineering function, containing the PCA-transformed datasets.
#         model (object): The machine learning model to be used for prediction (e.g., classifier or regressor).
#         reg (bool): If True, performs regression; otherwise, classification.
#         spread (float): The transaction cost or spread to be considered in strategy returns.
#         compressed_features (bool): If True, use PCA-transformed features; otherwise, use original features.

#     Returns:
#         float: The Sharpe ratio of the strategy based on predictions.
#     """

#     # Extract the data from the dictionary
#     X_train = data["X_train_pca"] if compressed_features else data["X_train_scaled"]
#     X_test = data["X_test_pca"] if compressed_features else data["X_test_scaled"]
#     X_val = data["X_val_pca"] if compressed_features else data["X_val_scaled"]
#     y_train_reg = data["y_train_reg"]
#     y_train_cla = data["y_train_cla"]
#     df = data["processed_df"]
#     split_train_test = data["split_train_test"]
#     split_test_valid = data["split_test_valid"]

#     # Fit the model on the training data
#     print("Model type:", type(model).__name__)

#     # Fit the model based on whether it is regression or classification
#     if hasattr(model, 'fit'):
#         # For sklearn models
#         if 'sklearn' in str(type(model)):
#             if reg:
#                 model.fit(X_train, y_train_reg)
#                 predictions = model.predict(np.concatenate((X_train, X_test, X_val), axis=0))
#             else:
#                 model.fit(X_train, y_train_cla)
#                 predictions = model.predict(np.concatenate((X_train, X_test, X_val), axis=0))

#         # For deep learning models (Pytorch, Keras, TensorFlow, etc.)
#         elif hasattr(model, 'predict'):
#             if reg:
#                 model.fit(X_train, y_train_reg, epochs=100, verbose=0)  # Specify epochs as needed
#                 predictions = model.predict(np.concatenate((X_train, X_test, X_val), axis=0)).flatten()
#             else:
#                 model.fit(X_train, y_train_cla, epochs=100, verbose=0)  # Specify epochs as needed
#                 predictions = model.predict(np.concatenate((X_train, X_test, X_val), axis=0))

#         else:
#             raise ValueError("Model does not have a 'fit' or 'predict' method.")
#     else:
#         raise ValueError("The provided model does not support fitting.")

#     # Clean the dataframe
#     # df = df.dropna()
#     # Impute missing values using IterativeImputer (Multiple Imputation by Chained Equations)
#     imputer = IterativeImputer(random_state=0)
#     clean_data = imputer.fit_transform(df)
#     df = pd.DataFrame(clean_data, columns=data["processed_df"].columns)


#     # Convert classification predictions to -1 (sell) and 1 (buy)
#     if not reg:
#         predictions = np.where(predictions == 0, -1, 1)
#         # predictions = np.argmax(predictions, axis=1)  # Assuming model returns probabilities

#     # Add predictions to the dataframe
#     df["prediction"] = predictions

#     # Compute the strategy returns (prediction * actual returns)
#     df["strategy"] = df["prediction"] * df["returns"]

#     # Select strategy returns only for the test set period
#     returns = df["strategy"].iloc[split_train_test:split_test_valid]

#     # Compute the Sharpe ratio of the strategy
#     sharpe_ratio = np.sqrt(252) * (returns.mean() - (spread / 100)) / returns.std()

#     return sharpe_ratio


## Find the best assets

In [ ]:

# def predictor(data: dict, model, reg: bool = True, spread: float = 0.035, compressed_features: bool = True) -> float:
#     """
#     Fits the model to the training data, makes predictions on the entire dataset, 
#     and computes the strategy's Sharpe ratio based on the predictions.
    
#     Args:
#         data (dict): The output of the features_engineering function, containing the PCA-transformed datasets.
#         model (object): The machine learning model to be used for prediction (e.g., classifier or regressor).
#         reg (bool): If True, performs regression; otherwise, classification.
#         spread (float): The transaction cost or spread to be considered in strategy returns.
#         compressed_features (bool): If True, use PCA-transformed features; otherwise, use original features.

#     Returns:
#         float: The Sharpe ratio of the strategy based on predictions.
#     """

#     # Extract the data from the dictionary
#     X_train = data["X_train_pca"] if compressed_features else data["X_train_scaled"]
#     X_test = data["X_test_pca"] if compressed_features else data["X_test_scaled"]
#     X_val = data["X_val_pca"] if compressed_features else data["X_val_scaled"]
#     y_train_reg = data["y_train_reg"]
#     y_train_cla = data["y_train_cla"]
#     df = data["processed_df"]
#     split_train_test = data["split_train_test"]
#     split_test_valid = data["split_test_valid"]

#     # Fit the model on the training data
#     print("Model type:", type(model).__name__)

#     # Fit the model on the training data
#     model.fit(X_train, y_train_cla)

#     # Clean the dataframe
#     # df = df.dropna()
#     # Impute missing values using IterativeImputer (Multiple Imputation by Chained Equations)
#     imputer = IterativeImputer(random_state=0)
#     clean_data = imputer.fit_transform(df)
#     df = pd.DataFrame(clean_data, columns=data["processed_df"].columns)
    
#     # Create predictions for the concatenated dataset (train, test, validation)
#     predictions = model.predict(np.concatenate((X_train, X_test, X_val), axis=0))

#     # Convert classification predictions to -1 (sell) and 1 (buy)
#     if not reg:
#         predictions = np.where(predictions == 0, -1, 1)
#         # predictions = np.argmax(predictions, axis=1)  # Assuming model returns probabilities

#     # Add predictions to the dataframe
#     df["prediction"] = predictions

#     # Compute the strategy returns (prediction * actual returns)
#     df["strategy"] = df["prediction"] * df["returns"]

#     # Select strategy returns only for the test set period
#     returns = df["strategy"].iloc[split_train_test:split_test_valid]

#     # Compute the Sharpe ratio of the strategy
#     sharpe_ratio = np.sqrt(252) * (returns.mean() - (spread / 100)) / returns.std()

#     return sharpe_ratio


In [ ]:
# # Import the necessary libraries
# from sklearn.svm import SVC
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.linear_model import LogisticRegression
# from tqdm import tqdm
# import concurrent.futures

# # Initialize models (only once)
# tree = DecisionTreeClassifier(max_depth=20)
# svr = SVC(C=1.5)
# lin = LogisticRegression()

# # Symbols list from final_assets
# symbols = final_assets["Symbol"]

# # To store results
# results_list = []

# # Define the function for processing each symbol
# def process_symbol(symbol):
#     print(symbol)
#     try:
#         # Retrieve data for the symbol
#         df = get_data(symbol, 3500, mt5.TIMEFRAME_D1).dropna()

#         # Perform feature engineering
#         processed_data = features_engineering(df)

#         # Decision Tree
#         sharpe_tree = predictor(processed_data, tree, reg=True, spread=spread_threshold)
#         result_tree = [symbol, "Tree", sharpe_tree, len(df)]

#         # SVM (Support Vector Classifier)
#         sharpe_svr = predictor(processed_data, svr, reg=False, spread=spread_threshold)
#         result_svr = [symbol, "SVR", sharpe_svr, len(df)]

#         # Logistic Regression
#         sharpe_linreg = predictor(processed_data, lin, reg=False, spread=spread_threshold)
#         result_linreg = [symbol, "LinReg", sharpe_linreg, len(df)]

#         return [result_tree, result_svr, result_linreg]

#     except KeyError:
#         return []
#     except Exception as e:
#         print(f"Issue during data importation or processing for symbol {symbol}: {e}")
#         return []

# # Use ThreadPoolExecutor to parallelize the process
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     # Execute the process for all symbols in parallel
#     results = list(tqdm(executor.map(process_symbol, symbols), total=len(symbols)))

# # Flatten the results list and filter out any empty results
# results_list = [result for sublist in results for result in sublist if sublist]

In [ ]:
# # Display the results after processing all symbols
# for result in results_list:
#     print(f"Symbol: {result[0]}, Model: {result[1]}, Sharpe Ratio: {result[2]:.4f}, Data Points: {result[3]}")


In [ ]:
# results = pd.DataFrame(results_list, columns=["Symbol", "Model", "Sharpe", "Length"])

# results

In [ ]:
# # Top 35 Symbols/Pairs
# results.sort_values(by="Sharpe", ascending=False).loc[results["Length"]>600].head(35)

### Combine the algorithms (Ensemble Methods)

#### List of symbols to process

In [ ]:
# # Top 5 Symbols/Pairs | ["US2000", "Bitcoin", "AUDUSD", "NAS100", "US500"]
# top_symbols = set(results.sort_values(by="Sharpe", ascending=False).loc[results["Length"]>600]['Symbol'].to_list()[:7])

# top_symbols

### Train and Save Models

In [ ]:
# import os
# import pickle
# from pathlib import Path
# from joblib import dump
# from sklearn.ensemble import VotingRegressor, VotingClassifier
# from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
# from sklearn.svm import SVR, SVC
# from sklearn.linear_model import LinearRegression, LogisticRegression

In [ ]:
# def voting(df, reg=True):
#     """Create a strategy using a voting method."""

#     processed_data = features_engineering(df)

#     # Extract the data from the dictionary
#     X_train_pca = processed_data["X_train_pca"]
#     X_test_pca = processed_data["X_test_pca"]
#     X_val_pca = processed_data["X_val_pca"]
#     y_train_reg = processed_data["y_train_reg"]
#     y_train_cla = processed_data["y_train_cla"]
    
#     # Initialize the models
#     if reg:
#         tree = DecisionTreeRegressor(max_depth=6)
#         svr = SVR(epsilon=1.5)
#         lin = LinearRegression()
#         vot = VotingRegressor(estimators=[
#             ('lr', lin), ("tree", tree), ("svr", svr)])
#     else:
#         tree = DecisionTreeClassifier(max_depth=6)
#         svr = SVC()
#         lin = LogisticRegression()

#         vot = VotingClassifier(estimators=[
#             ('lr', lin), ("tree", tree), ("svr", svr)])

#     # Train the model based on regression or classification task
#     if reg:
#         vot.fit(X_train_pca, y_train_reg)
#     else:
#         vot.fit(X_train_pca, y_train_cla)

#     # Remove missing values
#     # df = df.dropna()
#     # Impute missing values using IterativeImputer (Multiple Imputation by Chained Equations)
#     imputer = IterativeImputer(random_state=0)
#     clean_data = imputer.fit_transform(df)
#     df = pd.DataFrame(clean_data, columns=processed_data["processed_df"].columns)

#     # Create predictions for the entire dataset
#     df["prediction"] = vot.predict(np.concatenate((X_train_pca, X_test_pca, X_val_pca), axis=0))

#     # In case of classification, map the predictions to -1 and 1
#     if not reg:
#         df["prediction"] = np.where(df["prediction"] == 0, -1, 1)

#     # Compute strategy based on predictions
#     df["strategy"] = np.sign(df["prediction"]) * df["returns"]
#     df["low_strategy"] = np.where(df["prediction"] > 0, df["sLow"], -df["sHigh"])
#     df["high_strategy"] = np.where(df["prediction"] > 0, df["sHigh"], -df["sLow"])

#     return vot, df["strategy"], df["low_strategy"], df["high_strategy"]


In [ ]:
# Create a directory for models if it doesn't exist
model_dir = Path(os.getcwd()).parent / 'models'
# model_dir = os.path.join(os.getcwd(), 'models')
# os.makedirs(model_dir, exist_ok=True)  # Create directory if it doesn't exist

# Initialize lists to store results
results = pd.DataFrame()
low_assets = pd.DataFrame()
high_assets = pd.DataFrame()


# Function to compute returns and other metrics
def compute_metrics(df):
    """ Create custom metrics for strategy returns. """
    df["returns"] = ((df["close"] - df["close"].shift(1)) / df["close"])
    df["sLow"] = ((df["low"] - df["close"].shift(1)) / df["close"].shift(1))
    df["sHigh"] = ((df["high"] - df["close"].shift(1)) / df["close"].shift(1))
    return df.dropna()  # Remove missing values

# Function to load and process data
def get_and_process_data(symbol):
    """ Load data and apply feature engineering. """
    df = get_data(symbol, 3500, mt5.TIMEFRAME_D1).dropna()
    df = compute_metrics(df)  # Compute metrics
    return df


# for symbol in top_symbols:
#     print(f"Processing {symbol}...")
    
#     # Load and process the data
#     df = get_and_process_data(symbol)
    
#     # Compute the strategy using the voting function
#     vot, results[symbol], low_assets[symbol], high_assets[symbol] = voting(df, reg=False)

#     # Save the model using joblib
#     model_filename = os.path.join(model_dir, f"{symbol}_voting.joblib")
#     dump(vot, model_filename)
#     print(f"Model saved to {model_filename}")

# # Shutdown MT5
# mt5.shutdown()
